# Imports

Clone and install https://github.com/MTG/pycompmusic

Download data using `download-dunya.py`

In [ ]:
import json, os, sys
import pickle, csv
import time
import datetime
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px

import scipy
import librosa
from IPython.display import Audio as ipy_audio

In [ ]:
from quicktranscribe import tonic, pitch, wave

# Data

In [ ]:
data_dir = "/Users/neerajaabhyankar/Repos/icm-shruti-analysis/data-dunya-hindustani/"
track = "Omkar Dadarkar - Raag Bhoopali"

Metadata, Tonic, Pitch, Audio

In [ ]:
# metadata = tonic.read_metadata(data_dir + track + ".json")
ctonic = tonic.read_tonic(data_dir + track + ".ctonic.txt")
pitch_annotations, aps = pitch.read_pitch(data_dir + track + ".pitch.txt")

In [ ]:
y, sr = wave.get_audio(data_dir + track + ".mp3")

# Validating Pitch Annotations

In [ ]:
pv = pitch.PitchValidator(audio_array=y, sampling_rate=sr)
pv.set_annotation(pitch_annotations=pitch_annotations, annotation_rate=aps)

In [ ]:
# Find a representative sample
# 06:45 to 07:00
pv.play_sample(start_time=6*60+45, end_time=7*60)

In [ ]:
pv.validate_annotations(start_time=6*60+45, end_time=7*60)

# Manual Note Mode Finding

In [ ]:
pv.plot_annotations_hist()

Add counts to the list manual-shruti-peaks.txt

# TODO

NEXT:
1. refine the beat structure to 1/4
2. auto-detect tempo chunks, break by tempo, and apply the prototype
3. try some basic librosa pitch detection, see if it's better than the annotations, else just keep taking the mode over annotations
3. now we have a real time-series in φ

Select a sample

In [ ]:
# 06:45 to 07:00
start = 45*60+20
end = 45*60+32

y_small = y[start*sr:end*sr]
pa_small = pitch_annotations[start*aps:end*aps, 1]

In [ ]:
tempo, beat_frames = librosa.beat.beat_track(y=y_small, sr=sr)
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

Temp: collapse to known swars

In [ ]:
swar = {
    "D.": 0.8362, "S": 0.997, "R": 1.11896, "G": 1.2565, "P": 1.501
}
swars = np.array(list(swar.values()))*tonic
swars = np.concatenate((swars/2, swars, swars*2))

In [ ]:
swar_small = []
for pa in pa_small:
    if pa == 0:
        swar_small.append(0)
    else:
        idx = (np.abs(swars - pa)).argmin()
        swar_small.append(swars[idx])

# plt.plot(range(len(pa_small)), pa_small)
# plt.plot(range(len(swar_small)), swar_small)

In [ ]:
plt.plot(range(len(pa_small)), pa_small)
plt.plot(range(len(swar_small)), swar_small)

Note: the following won't scale! Just for now, looking at how the mode sounds

In [ ]:
beat_swars = []
y_swar_small = []
for beat_idx in range(1, len(beat_times)):
    beat_start = beat_times[beat_idx-1]
    beat_end = beat_times[beat_idx]
    swars_beat = swar_small[int(beat_start*aps):int(beat_end*aps)]
    
    idx = (np.abs(swars - scipy.stats.mode(swars_beat).mode)).argmin()
    beat_swars.append(swars[idx])
    
    tone = librosa.tone(swars[idx], sr=sr, length=(beat_end-beat_start)*sr)
    y_swar_small += tone.tolist()

    

In [ ]:
print("beat-wise swar-collapsed waveform")
ipy_audio(data=y_swar_small, rate=sr)

In [ ]:
print("original sample")
ipy_audio(data=y_small, rate=sr)